In [4]:
from datetime import datetime
from pathlib import Path
import gsw
import numpy as np
import pandas as pd
import xarray as xr
import pyarrow.parquet as pq
import requests as rq
import os
from urllib.parse import urljoin, urlencode
import argo_tools as at
import argopy
from argopy import DataFetcher as ArgoDataFetcher
local_gdac = '/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData'
#Path.mkdir('/vortexfs1/share/boom/projects/n2o/pq')
outdir_nc = '/vortexfs1/share/boom/data/nc2pqt_test/GDAC/'
outdir_pqt = '/vortexfs1/share/boom/data/nc2pqt_test/PQT/' #'/vortexfs1/share/boom/projects/n2o/pq'
# argopy.set_options(mode='expert',src='gdac',ftp=local_gdac)
from pprint import pprint
import warnings
import glob
import psutil

In [5]:
wmos, df2, wmo_fp = at.argo_gdac(lat_range=[0,20],lon_range=[-30,-20],save_to=outdir_nc, download_individual_profs=False, dryrun=False, skip_downloads=False)

/vortexfs1/home/enrico.milanese/projects/ARGO/nc2parquet/argo_tools.py:50: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  gdac_index = pd.read_csv(gdac_path,delimiter=',',header=8,parse_dates=['date','date_update'],


>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/1901467_Sprof.nc.
>>> File 1901467_Sprof.nc already exists. Leaving current version.
>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/1901468_Sprof.nc.
>>> File 1901468_Sprof.nc already exists. Leaving current version.
>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/1901668_Sprof.nc.
>>> File 1901668_Sprof.nc already exists. Leaving current version.
>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/4901283_Sprof.nc.
>>> File 4901283_Sprof.nc already exists. Leaving current version.
>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/4901460_Sprof.nc.
>>> File 4901460_Sprof.nc already exists. Leaving current version.
>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/6901174_Sprof.nc.
>>> File 6901174_Sprof.nc already exists. Leaving current version.
>>>> Destination file: /vortexfs1/share/boom/data/nc2pqt_test/GDAC/6901175_Sprof.nc.
>>>

In [6]:
pprint(wmo_fp)

['1901467_Sprof.nc',
 '1901468_Sprof.nc',
 '1901668_Sprof.nc',
 '4901283_Sprof.nc',
 '4901460_Sprof.nc',
 '6901174_Sprof.nc',
 '6901175_Sprof.nc',
 '1900650_Sprof.nc',
 '1900651_Sprof.nc',
 '1900652_Sprof.nc',
 '1900943_Sprof.nc',
 '1901360_Sprof.nc',
 '1901361_Sprof.nc',
 '1901363_Sprof.nc',
 '1901364_Sprof.nc',
 '1901365_Sprof.nc',
 '3900281_Sprof.nc',
 '3902122_Sprof.nc',
 '3902123_Sprof.nc',
 '6900524_Sprof.nc',
 '6900525_Sprof.nc',
 '6900628_Sprof.nc',
 '6900629_Sprof.nc',
 '6900630_Sprof.nc',
 '6900631_Sprof.nc',
 '6900632_Sprof.nc',
 '6900889_Sprof.nc',
 '6900890_Sprof.nc',
 '6902984_Sprof.nc',
 '6903067_Sprof.nc',
 '6903091_Sprof.nc',
 '6903876_Sprof.nc',
 '6903877_Sprof.nc',
 '6904134_Sprof.nc',
 '6904139_Sprof.nc',
 '7900559_Sprof.nc',
 '7900560_Sprof.nc',
 '7900561_Sprof.nc']


#### Importing iterators from [`itertools`](https://docs.python.org/3/library/itertools.html)
* `islice` returns selected elements from iterable
* `batched` split the iterable object into tuples of prescribed length _n_ (if `length(iterable)%n~=0`, the last tuple is shorter than _n_).

In [7]:
import sys
from itertools import islice

if sys.version_info >= (3, 12):
    from itertools import batched
else:
    try:
        from more_itertools import batched
    except ImportError:
        def batched(iterable, chunk_size):
            iterator = iter(iterable)
            while chunk := tuple(islice(iterator, chunk_size)):
                yield chunk

In [8]:
VARS = ['JULD','CYCLE_NUMBER','PLATFORM_NUMBER','LATITUDE','LONGITUDE',
 'PRES',
 'PRES_QC',
 'PRES_ADJUSTED',
 'PRES_ADJUSTED_QC',
 'PRES_ADJUSTED_ERROR',
 'TEMP',
 'TEMP_QC',
 'TEMP_dPRES',
 'TEMP_ADJUSTED',
 'TEMP_ADJUSTED_QC',
 'TEMP_ADJUSTED_ERROR',
 'PSAL',
 'PSAL_QC',
 'PSAL_dPRES',
 'PSAL_ADJUSTED',
 'PSAL_ADJUSTED_QC',
 'PSAL_ADJUSTED_ERROR',
 'DOXY',
 'DOXY_QC',
 'DOXY_dPRES',
 'DOXY_ADJUSTED',
 'DOXY_ADJUSTED_QC',
 'DOXY_ADJUSTED_ERROR',
 'CHLA',
 'CHLA_QC',
 'CHLA_dPRES',
 'CHLA_ADJUSTED',
 'CHLA_ADJUSTED_QC',
 'CHLA_ADJUSTED_ERROR',
 'BBP700',
 'BBP700_QC',
 'BBP700_dPRES',
 'BBP700_ADJUSTED',
 'BBP700_ADJUSTED_QC',
 'BBP700_ADJUSTED_ERROR',
 'CDOM',
 'CDOM_QC',
 'CDOM_dPRES',
 'CDOM_ADJUSTED',
 'CDOM_ADJUSTED_QC',
 'CDOM_ADJUSTED_ERROR',
 'PH_IN_SITU_TOTAL',
 'PH_IN_SITU_TOTAL_QC',
 'PH_IN_SITU_TOTAL_dPRES',
 'PH_IN_SITU_TOTAL_ADJUSTED',
 'PH_IN_SITU_TOTAL_ADJUSTED_QC',
 'PH_IN_SITU_TOTAL_ADJUSTED_ERROR',
 'NITRATE',
 'NITRATE_QC',
 'NITRATE_dPRES',
 'NITRATE_ADJUSTED',
 'NITRATE_ADJUSTED_QC',
 'NITRATE_ADJUSTED_ERROR']

### Parallel 

The conversion from `nc` to `parquet` is parallelized. All you need to do is comment/uncomment the appropriate line in the next cell, selecting if you want to process the files that you just downloaded (first line) or other files (second line, modify to include your desired path).
The new parquet files will be stored in the directory `outdir_pqt` that you specified earlier.

In [9]:
flist = [outdir_nc + s for s in wmo_fp]
#flist = glob.glob("/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/*/*_Sprof.nc")

In [12]:
%%time

import multiprocessing

def xr2pqt(rank,files_list,loop_id):
    df_list = []
    df_memory = 0
    counter = 0
    rank_str = "#" + str(rank) + ": "
    nb_files = len(files_list)
    for argo_file in files_list:
        counter += 1
        if counter%10==0:
            print(rank_str + "processing file " + str(counter) + " of " + str(nb_files))
            
        try:
            ds = xr.load_dataset(argo_file, engine="argo") #loading into memory the profile
        except:
            print(rank_str + 'Failed on ' + str(argo_file))
        
        invars = list(set(VARS) & set(list(ds.data_vars)))
        df = ds[invars].to_dataframe()
        df_memory += df.memory_usage().sum()/(10**9) # tracking memory usage (in GB)
        df_list.append(df)

    # store to parquet once a large enough dataframe has been created
    
    print(rank_str + "Storing to parquet...")
    print(rank_str + "Filesize: " + "{:.2f}".format(df_memory) + " GB")
    
    df_list = pd.concat(df_list)

    parquet_filename = outdir_pqt + "test_parquet_" + str(rank) + "_" + str(loop_id) + ".parquet"
    df_list.to_parquet(parquet_filename)
    print(rank_str + str(parquet_filename) + " stored.")

    df_list = []
    df_memory = 0
    
############################################################################################################

def poolParams(flist):
    size_flist = []
    for f in flist:
        size_flist.append( os.path.getsize(f)/1024**2 ) #size in MB
    
    size_tot = sum(size_flist)
    NPROC = int(np.ceil(size_tot/400)) # Empirically, 300 MB of .nc files seems a good trade-off
    size_per_proc = size_tot/NPROC

    print('')
    print('Size per processor (MB)')
    print(size_per_proc)
    print('')
    
    ids_sort = np.argsort(np.array(size_flist))
    
    chunks_ids = []
    x = np.copy(ids_sort)
    
    for j in range(NPROC):
        chunk_ids = []
        chunk_size = 0
        while ((chunk_size<size_per_proc) and (len(x) > 0)):
            if len(chunk_ids)%2 == 0:
                chunk_ids.append(x[-1])
                x = x[:-1]
            else:
                chunk_ids.append(x[0])
                x = x[1:]
            chunk_size = sum(np.asarray(size_flist)[chunk_ids])
        print(chunk_size)
        chunks_ids.append(chunk_ids)
    
    if len(x) > 0:
        warnings.warn(str(len(x)) + " files have not been assigned to a processor.")
    
    print('')
    chunks=[]
    skip_proc = 0
    total_memory = 0
    for j,chunk_ids in enumerate(chunks_ids):
        print('Size in processor ' + str(j) + ' (MB):')
        size_proc = sum(np.asarray(size_flist)[chunk_ids])
        total_memory += size_proc
        print(size_proc)
        if size_proc == 0:
            skip_proc += 1
            continue
        chunk = [flist[k] for k in chunk_ids]
        chunks.append(chunk)

    NPROC -= skip_proc
        
    print('')
    print("Using " + str(NPROC) + " processors")
    
    return NPROC, chunks

############################################################################################################

print("Processing " + str(len(flist)) + " files.")

NPROC, chunks = poolParams(flist)

# fixing max nb of processes to prevent bottleneck likely due to I/O on disk queing operations and filling up the memory
MAXPROC = 12

if NPROC > MAXPROC:
    print("Estimated number of processors might create bottleneck issues. Forcing to use " + str(MAXPROC) + " processors at a time.")
    # force to use at most MAXPROC processes, by looping over chunks
    full_loops = NPROC//MAXPROC  #nb of loops to use at most MAXPROC
    RESPROC = NPROC%MAXPROC   #nb of residual processors after the loops
    pool_obj = multiprocessing.Pool(processes=MAXPROC)
    i_start = 0
    i_end   = 0
    for full_loop in range(full_loops):
        i_start = MAXPROC*full_loop
        i_end   = MAXPROC*(full_loop+1)
        pool_obj.starmap(xr2pqt, [(rank, chunk, full_loop) for rank, chunk in enumerate(chunks[i_start:i_end])] )
    pool_obj.close()

    # multiprocessing across residual processor pool with NPROC<MAXPROC
    if RESPROC > 0:
        pool_obj = multiprocessing.Pool(processes=RESPROC)
        pool_obj.starmap(xr2pqt, [(rank, chunk, full_loop+1) for rank, chunk in enumerate(chunks[(i_end+1):])] )
        pool_obj.close()

elif NPROC > 1:
    pool_obj = multiprocessing.Pool(processes=NPROC)
    pool_obj.starmap(xr2pqt, [(rank, chunk, 0) for rank, chunk in enumerate(chunks)] )
    pool_obj.close()

else:
    xr2pqt(0,flist,0)

Processing 38 files.

Size per processor (MB)
372.1313171386719

372.1313171386719

Size in processor 0 (MB):
372.1313171386719

Using 1 processors
#0: processing file 10 of 38
#0: processing file 20 of 38
#0: processing file 30 of 38
#0: Storing to parquet...
#0: Filesize: 1.91 GB
#0: /vortexfs1/share/boom/data/nc2pqt_test/PQT/test_parquet_0_0.parquet stored.
CPU times: user 1min 35s, sys: 5.19 s, total: 1min 40s
Wall time: 1min 40s


### Test direct subsetting from parquet directory

In [ ]:
#t = pq.read_table(Path(outdir),filters=[('PRES', '<', 20),('LATITUDE', '<', 21.1),('LATITUDE', '>', 21)])
t = pq.read_table(Path(outdir),filters=[('PLATFORM_NUMBER', '==', 1902304)])
df = t.to_pandas()
df
#ds = xr.Dataset.from_dataframe(df)
#ds

### Example loading Sprof from snapshot
```
ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc')
df = ds.to_dataframe()
```

In [ ]:

from dotenv import load_dotenv
load_dotenv()
import os
from pyarrow import fs
s3 = fs.S3FileSystem(region='us-east-1')


In [ ]:
s3

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq
from pyarrow import Table

ds = xr.load_dataset('/vortexfs1/share/boom/data/gdac_snapshot/202403-ArgoData/dac/aoml/1902304/1902304_Sprof.nc',engine="argo")
df = ds[['DOXY','PRES','NITRATE','PLATFORM_NUMBER']].to_dataframe()

s3_filepath = 'data.parquet'

pq.write_to_dataset(
    Table.from_pandas(df),
    s3_filepath,
    filesystem=s3,
    use_dictionary=True,
    compression="snappy",
    version="2.4",
)



### single threaded

In [ ]:

CHUNK_SZ = 100
VARS = ['JULD','LATITUDE','LONGITUDE','PRES','PRES_ADJUSTED','DOXY_ADJUSTED','DOXY_ADJUSTED_QC','NITRATE','NITRATE_ADJUSTED','PSAL','TEMP','CYCLE_NUMBER','PLATFORM_NUMBER']
for chunk in batched(wmo_fp,CHUNK_SZ):
    dflist = []
    for line in chunk:  
        fn = (line.split('/')[1] + "_Sprof.nc")
        fpath = Path(local_gdac) / "dac" / line / fn
        try:
            ds = xr.load_dataset(fpath)
        except:
            print(fpath)
        invars = list(set(VARS) & set(list(ds.data_vars)))
        df = ds[invars].to_dataframe()
        dflist.append(df)
    print(fpath)
    df = pd.concat(dflist)
    df.to_parquet('pq/test' + line.split('/')[1] + ".parquet",coerce_timestamps='us',allow_truncated_timestamps=True)
        
        